In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install --upgrade transformers
!pip install datasets
!pip install evaluate
!pip install bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.5 MB/s eta 0:00:00


In [ ]:
import evaluate
import torch
from tqdm import tqdm
import pandas as pd
from datasets import Dataset
from transformers import BartTokenizer, BartForConditionalGeneration, TrainingArguments, Trainer
import torch


In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/DS Bootcamp/Automated Meeting Summarizer/cleaned_train.csv")
val_df = pd.read_csv('/content/drive/MyDrive/DS Bootcamp/Automated Meeting Summarizer/cleaned_val.csv')
test_df = pd.read_csv('/content/drive/MyDrive/DS Bootcamp/Automated Meeting Summarizer/cleaned_test.csv')

In [ ]:
train_df = train_df[['cleaned_dialogue', 'summary']]
val_df = val_df[['cleaned_dialogue', 'summary']]
test_df = test_df[['cleaned_dialogue', 'summary']]

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

In [ ]:
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

In [ ]:
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples['cleaned_dialogue'],
        max_length=512,
        truncation=True,
        padding='max_length'
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples['summary'],
            max_length=128,
            truncation=True,
            padding='max_length'
        )
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [ ]:
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_val = val_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/11863 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/230 [00:00<?, ? examples/s]

In [ ]:
model = BartForConditionalGeneration.from_pretrained(model_name)


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=3e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=1000,
    report_to=[]
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
)

trainer.train()


/tmp/ipython-input-2802915120.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
1000,0.568600
2000,0.431900
3000,0.409700
4000,0.400700
5000,0.381700


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=5932, training_loss=0.4262207555995949, metrics={'train_runtime': 4507.7238, 'train_samples_per_second': 2.632, 'train_steps_per_second': 1.316, 'total_flos': 1.2854180947427328e+16, 'train_loss': 0.4262207555995949, 'epoch': 1.0})

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        

In [ ]:
def generate_summary(text):
    inputs = tokenizer([text], max_length=512, truncation=True, return_tensors='pt')

    inputs = {key: val.to(device) for key, val in inputs.items()}

    summary_ids = model.generate(
        inputs['input_ids'],
        max_length=126,
        min_length=5,
        num_beams=4,
        early_stopping=True
    )
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
sample = test_df.iloc[0]['cleaned_dialogue']
print("Original Dialogue:\n", sample)
print("\nGenerated Summary:\n", generate_summary(sample))
print("\nOriginal Summary:\n", test_df.iloc[0]['summary'])

Original Dialogue:
 speaker youre finally took long speaker got stuck traffic terrible traffic jam near carrefour intersection speaker always rather congested rush hour maybe try find different route get home speaker dont think avoided honest speaker perhaps would better started taking public transport system work speaker think something ill consider public transport system pretty good speaker would better environment speaker know feel bad much car adding pollution problem city speaker taking subway would lot le stressful driving well speaker problem im going really miss freedom car speaker well nicer outside start biking work give much freedom car usually provides speaker thats true could certainly use exercise speaker going quit driving work speaker yes good environment

Generated Summary:
 #Person1# and #Person2# are stuck in a terrible traffic jam near the Carrefour intersection. They decide to start taking the public transport system to work. They both think it's better for the en

## **BERT Score**

In [ ]:
bertscore = evaluate.load("bertscore")

In [ ]:
predictions = []
references = []

for i in tqdm(range(50)):
    dialogue = test_df.iloc[i]['cleaned_dialogue']
    reference = test_df.iloc[i]['summary']

    inputs = tokenizer([dialogue], max_length=512, truncation=True, return_tensors='pt')

    summary_ids = model.generate(
        inputs['input_ids'].to(device),
        max_length=80,
        min_length=10,
        num_beams=4,
        early_stopping=True
    )

    generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    predictions.append(generated_summary)
    references.append(reference)

results = bertscore.compute(predictions=predictions, references=references, lang="en")


100%|██████████| 50/50 [00:27<00:00,  1.82it/s]


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(f"Precision: {sum(results['precision'])/len(results['precision']):.4f}")
print(f"Recall:    {sum(results['recall'])/len(results['recall']):.4f}")
print(f"F1 Score:  {sum(results['f1'])/len(results['f1']):.4f}")

Precision: 0.9155
Recall:    0.9153
F1 Score:  0.9153


In [ ]:
from sentence_transformers import SentenceTransformer, util

## **Cosine Similarity**

In [ ]:
model_st = SentenceTransformer('all-MiniLM-L6-v2')  # lightweight and accurate

predictions = []
references = []

# Generate summaries for the first 50 test samples
for i in tqdm(range(50)):
    dialogue = test_df.iloc[i]['cleaned_dialogue']
    reference = test_df.iloc[i]['summary']

    # Tokenize the input
    inputs = tokenizer([dialogue], max_length=512, truncation=True, return_tensors='pt')

    # Generate the summary
    summary_ids = model.generate(
        inputs['input_ids'].to(device),
        max_length=80,
        min_length=10,
        num_beams=4,
        early_stopping=True
    )

    # Decode the generated summary
    generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    predictions.append(generated_summary)
    references.append(reference)

# Compute sentence embeddings
pred_embeddings = model_st.encode(predictions, convert_to_tensor=True)
ref_embeddings = model_st.encode(references, convert_to_tensor=True)

# Compute cosine similarity for each pair
cosine_scores = util.cos_sim(pred_embeddings, ref_embeddings)

# Average diagonal values (each summary compared to its reference)
average_score = cosine_scores.diag().mean().item()



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

100%|██████████| 50/50 [00:28<00:00,  1.76it/s]

Average Cosine Similarity: 0.7518


In [ ]:
print(f"Average Cosine Similarity: {average_score:.4f}")

Average Cosine Similarity: 0.7518


## **Saving Model**

In [ ]:
model.save_pretrained("/content/drive/MyDrive/bart_large_cnn_BERT")
tokenizer.save_pretrained("/content/drive/MyDrive/bart_large_cnn_BERT")

('/content/drive/MyDrive/bart_large_cnn/tokenizer_config.json',
 '/content/drive/MyDrive/bart_large_cnn/special_tokens_map.json',
 '/content/drive/MyDrive/bart_large_cnn/vocab.json',
 '/content/drive/MyDrive/bart_large_cnn/merges.txt',
 '/content/drive/MyDrive/bart_large_cnn/added_tokens.json')